In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
savePath = "s3://corpus-2/model/ad_biLSTM"

In [12]:
def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])

        for i in range(num):
            np.random.shuffle(train_data[i])
            train_data[i] = np.asarray(train_data[i])

        np.random.shuffle(train_data)   
        return train_data
    
    
    train_data = []
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(model, train_data, split_type):
    
    print(split_type)
    
    train_acc = []
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(tx[test_i],ty[test_i])
            train_acc.append(model_train(model,train_x, train_y,split_type,fold_id))
            test_acc.append(model_evaluate(model,test_x, test_y,split_type,fold_id,categories))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_acc.append(model_train(model,tx[train_i], ty[train_i],split_type,fold_id))
            test_acc.append(model_evaluate(model,tx[test_i], ty[test_i],split_type,fold_id,categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc


def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, model, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {model.inputX: x_batch1, model.inputY: y_batch1, model.dropoutKeepProb: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(model, x_train, y_train, split_type, fold_id):
    
    save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    start_time = time.time()
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
    flag = False

    for epoch in range(num_epochs):  # 20
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {model.inputX: x_batch, model.inputY: y_batch, model.dropoutKeepProb: dropout_keep_prob}
            session.run(model.trainOp, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.dropoutKeepProb] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
                if acc_train > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_train
                    last_improved = total_batch
                    saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''

                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
                print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return best_acc_train


def model_evaluate(model, x_test, y_test, split_type, fold_id, categories, batch_size=64):
        
    save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 读取保存的模型
    session = tf.Session()
    saver.restore(sess=session, save_path=save_path)
    start_time = time.time()
    print('Testing...')
    loss_test, acc_test = evaluate(session, model, x_test, y_test, model.loss, model.acc, batch_size)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    test_data_len = len(x_test)
    test_num_batch = int((test_data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)  # 获得类别
    y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

    for i in range(test_num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, test_data_len)
        feed_dict = {
            model.inputX: x_test[start_id:end_id],
            model.dropoutKeepProb: 1.0
        }
        y_test_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
    '''
    sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
        y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
        y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
        labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
        target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
        原文链接：https://blog.csdn.net/akadiao/article/details/78788864
    '''

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
    '''
    混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
    这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
    https://blog.csdn.net/u011734144/article/details/80277225
    '''
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)
    session.close()

    return acc_test

In [5]:
seq_length = 41

In [6]:
vocab, embd = loadGloVe(vocabPath, 50)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

Loading GloVe!
Completed!


In [7]:
word_to_id = dict(zip(vocab, range(vocab_size)))

In [8]:
len(embedding),embedding_dim,vocab_size

(400000, 100, 400000)

In [9]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

In [10]:
def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [11]:
# 构建adversarailLSTM模型
class AdversarailLSTM(object):

    def __init__(self, wordEmbedding):
        # 定义输入
        self.inputX = tf.placeholder(tf.int32, [None, seq_length], name='inputX')
        self.inputY = tf.placeholder(tf.int32, [None, num_classes], name='inputY')

        self.dropoutKeepProb = tf.placeholder(tf.float64, name='keep_prob')

        # 词嵌入层
        with tf.name_scope("wordEmbedding"):
            self.embeddedWords = tf.nn.embedding_lookup(wordEmbedding, self.inputX)

        # 计算softmax交叉熵损失
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                # self.y_pred_cls = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                self.y_pred_cls = tf.argmax(tf.nn.softmax(self.predictions),1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）
                # losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)

        
        with tf.name_scope("perturloss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                print("perturbSize:{}".format(perturWordEmbedding))
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                # perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)

        self.loss = loss + perturLoss
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(self.loss)
        # 将梯度应用到变量下，生成训练器
        self.trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.inputY, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        # self.loss = loss
        
        
    def _Bi_LSTMAttention(self, embeddedWords):
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向网络结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                    # 定义反向网络结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)
            # 采用动态rnn，可以动态地输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元组(output_fw, output_bw), 其中两个元素的维度都是[batch_size, max_time, hidden_size], fw和bw的hiddensize一样
            # self.current_state是最终的状态，二元组(state_fw, state_bw), state_fw=[batch_size, s], s是一个元组(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm,
                                                                          self.embeddedWords, dtype=tf.float64,
                                                                          scope="bi-lstm" + str(idx))

        # 在bi-lstm+attention论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到attention的输出
            output = self.attention(H)
            outputSize = hiddenSizes[-1]
            print("outputSize:{}".format(outputSize))

        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW", dtype=tf.float64,
                shape=[outputSize, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())

            outputB = tf.Variable(tf.constant(0.1, dtype=tf.float64, shape=[num_classes]), name="outputB")

            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")

            return predictions

    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层lstm神经元的数量
        hiddenSize = hiddenSizes[-1]

        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1, dtype=tf.float64))

        # 对bi-lstm的输出用激活函数做非线性转换
        M = tf.tanh(H)

        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size], 计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1], 每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))

        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, seq_length])

        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)

        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, seq_length, 1]))

        # 将三维压缩成二维sequeezeR = [batch_size, hissen_size]
        sequeezeR = tf.squeeze(r)

        sentenceRepren = tf.tanh(sequeezeR)

        # 对attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)

        return output

    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)

        var = tf.matmul(weights, powWordEmbedding)
        stddev = tf.sqrt(1e-6 + var)

        return (wordEmbedding - mean) / stddev

    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, epsilon)
        # print("perturbSize:{}".format(embedded+perturb))
        return embedded + perturb

    def _scaleL2(self, x, norm_length):
        # shape(x) = [batch, num_step, d]
        # divide x by max(abs(x)) for a numerically stable L2 norm
        # 2norm(x) = a * 2norm(x/a)
        # scale over the full sequence, dim(1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [13]:
hiddenSizes = [128]  # 定义LSTM的隐藏层（一层，128个神经元）
epsilon = 5

num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 50
batch_size = 64
print_per_batch = 30  # 每多少轮输出一次结果


lstm = AdversarailLSTM(embedding)
saver = tf.train.Saver()

outputSize:128
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


perturbSize:Tensor("perturloss/Bi-LSTM/add_2:0", shape=(?, 41, 100), dtype=float64)
outputSize:128


In [18]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, split_type))
    

random
14035 41 41
Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    4.5, Train Acc:  18.75%, Time: 0:00:15 *
Iter:     60, Train Loss:    4.1, Train Acc:  32.81%, Time: 0:00:25 *
Iter:     90, Train Loss:    3.4, Train Acc:  50.00%, Time: 0:00:35 *
Iter:    120, Train Loss:    2.7, Train Acc:  57.81%, Time: 0:00:46 *
Iter:    150, Train Loss:    2.6, Train Acc:  57.81%, Time: 0:00:52 
Iter:    180, Train Loss:    2.0, Train Acc:  73.44%, Time: 0:01:03 *
Epoch: 2
Iter:    210, Train Loss:    2.6, Train Acc:  57.81%, Time: 0:01:09 
Iter:    240, Train Loss:    2.2, Train Acc:  62.50%, Time: 0:01:15 
Iter:    270, Train Loss:    1.4, Train Acc:  81.25%, Time: 0:01:25 *
Iter:    300, Train Loss:    2.4, Train Acc:  65.62%, Time: 0:01:31 
Iter:    330, Train Loss:    1.7, Train Acc:  76.56%, Time: 0:01:37 
Iter:    360, Train Loss:    1.2, Train Acc:  79.69%, Time: 0:01:43 
Iter:    390, Train Loss:    1.7, Train Acc:  70.31%, Time: 0:01:49 
Epoch: 3
Iter:    420, T

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ad_biLSTM/random/1/1


Testing...
Test Loss:   0.72, Test Acc:  89.53%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.86      0.94      0.90       140
                   Filter       0.77      0.90      0.83       138
    Compute Derived Value       0.89      0.91      0.90       158
            Find Extremum       0.90      0.82      0.86       162
                     Sort       0.92      0.95      0.94       115
          Determine Range       0.87      0.88      0.88       138
Characterize Distribution       0.97      0.90      0.93       140
           Find Anomalies       0.92      0.80      0.86       128
                  Cluster       0.98      0.91      0.94       134
                Correlate       0.93      0.95      0.94       151

                micro avg       0.90      0.90      0.90      1404
                macro avg       0.90      0.90      0.90      1404
             weighted avg       0.90      0.9

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ad_biLSTM/random/2/2


Testing...
Test Loss:   0.59, Test Acc:  91.31%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.90      0.98      0.94       132
                   Filter       0.89      0.93      0.91       139
    Compute Derived Value       0.90      0.90      0.90       150
            Find Extremum       0.96      0.92      0.94       171
                     Sort       0.96      0.96      0.96       115
          Determine Range       0.86      0.92      0.89       122
Characterize Distribution       0.91      0.87      0.89       131
           Find Anomalies       0.90      0.84      0.87       157
                  Cluster       0.90      0.95      0.92       145
                Correlate       0.95      0.87      0.91       142

                micro avg       0.91      0.91      0.91      1404
                macro avg       0.91      0.91      0.91      1404
             weighted avg       0.91      0.9

...

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ad_biLSTM/table/4/4


Testing...
Test Loss:    3.2, Test Acc:  61.09%
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.46      0.38      0.42        95
                   Filter       0.37      0.71      0.48       106
    Compute Derived Value       0.66      0.28      0.39       178
            Find Extremum       0.82      0.72      0.77       229
                     Sort       0.72      0.87      0.79        61
          Determine Range       0.60      0.64      0.62       108
Characterize Distribution       0.57      0.72      0.64        72
           Find Anomalies       0.56      0.91      0.69       108
                  Cluster       0.85      0.65      0.74        72
                Correlate       0.72      0.46      0.56       107

                micro avg       0.61      0.61      0.61      1136
                macro avg       0.63      0.63      0.61      1136
             weighted avg       0.65      0.6

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ad_biLSTM/table/5/5


Testing...


InvalidArgumentError: In[0] is not a matrix
	 [[Node: loss/Bi-LSTM/output/predictions/MatMul = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss/Bi-LSTM/Attention/dropout/mul, Bi-LSTM/outputW/read)]]

Caused by op 'loss/Bi-LSTM/output/predictions/MatMul', defined at:
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/ipyker

In [19]:
len(train_data)

37

In [61]:
train_acc = []
test_acc = []
fold_id = 0

split_type = "table"
tx = []
ty = []
for ti in train_data:
    x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
    tx.append(x_train)
    ty.append(y_train)

tx = np.asarray(tx)
ty = np.asarray(ty)

print(len(tx),len(tx[0]),len(tx[1]),len(tx[0][0]))

for train_i, test_i in kf.split(tx):
    fold_id += 1
    if fold_id >= 5:
        print("Fold: ", fold_id)
        train_x, train_y = mergeData(tx[train_i],ty[train_i])
        test_x, test_y = mergeData(tx[test_i],ty[test_i])
        train_acc.append(model_train(lstm,train_x, train_y,split_type,fold_id))
        test_acc.append(model_evaluate(lstm,test_x, test_y,split_type,fold_id,categories))

37 322 826 41
Fold:  5
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    4.4, Train Acc:  17.19%, Time: 0:00:16 *
Iter:     60, Train Loss:    4.0, Train Acc:  23.44%, Time: 0:00:28 *
Iter:     90, Train Loss:    3.6, Train Acc:  40.62%, Time: 0:00:40 *
Iter:    120, Train Loss:    3.0, Train Acc:  45.31%, Time: 0:00:51 *
Iter:    150, Train Loss:    2.4, Train Acc:  65.62%, Time: 0:01:04 *
Iter:    180, Train Loss:    2.1, Train Acc:  73.44%, Time: 0:01:16 *
Epoch: 2
Iter:    210, Train Loss:    2.2, Train Acc:  70.31%, Time: 0:01:21 
Iter:    240, Train Loss:    1.7, Train Acc:  71.88%, Time: 0:01:27 
Iter:    270, Train Loss:    1.8, Train Acc:  71.88%, Time: 0:01:33 
Iter:    300, Train Loss:    2.1, Train Acc:  67.19%, Time: 0:01:39 
Iter:    330, Train Loss:    1.8, Train Acc:  73.44%, Time: 0:01:44 
Iter:    360, Train Loss:    2.1, Train Acc:  70.31%, Time: 0:01:50 
Iter:    390, Train Loss:    1.3, Train Acc:  84.38%, Time: 0:02:01 *
Epoch: 3
Iter:    420, Train

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/ad_biLSTM/table/5/5


Testing...


InvalidArgumentError: In[0] is not a matrix
	 [[Node: loss/Bi-LSTM/output/predictions/MatMul = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss/Bi-LSTM/Attention/dropout/mul, Bi-LSTM/outputW/read)]]

Caused by op 'loss/Bi-LSTM/output/predictions/MatMul', defined at:
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/ipyker

In [40]:
len(test_x),len(test_x[0]),len(test_y),len(test_y[0])

(1136, 41, 1136, 10)

In [43]:
test_x[0],test_x[1],test_y[0],test_y[1]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2237,
        2867, 4249,   33,  356,  197,  109, 1145,  188], dtype=int32),
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   102,    32,    77,  1145,
           12,    32, 13773,   173,   188], dtype=int32),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]))

In [45]:
len(test_x),len(test_x[0]),len(test_y),len(test_y[0])

(1153, 41, 1153, 10)

In [53]:
test_x[-1],test_y[-1],test_x[1152]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    6,    0,   62,  778,
          42,  123,   40,    0,   96, 8786,  543,  188], dtype=int32),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    6,    0,   62,  778,
          42,  123,   40,    0,   96, 8786,  543,  188], dtype=int32))